In [8]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import folium
import json
import streamlit as st

ImportError: Pandas requires version '3.0.0' or newer of 'jinja2' (version '2.11.3' currently installed).

In [ ]:
###Inladen API - kijk naar country code en maxresults
response = requests.get('https://api.openchargemap.io/v3/poi?output=json&countrycode=NL&maxresults=100&compact=true&verbose=false&key=17194855-0114-467b-805a-489c9df1e25f')
response.status_code

In [ ]:
###Omzetten naar dictionary
json = response.json()
json

In [ ]:
###Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
Laadpalen = pd.json_normalize(json)
#Daarna nog handmatig kijken welke kolommen over zijn in dit geval Connections
#Kijken naar eerst laadpaal op de locatie
#Kan je uitpakken middels:
df4 = pd.json_normalize(Laadpalen.Connections)
df5 = pd.json_normalize(df4[0])
df5.head()
###Bestanden samenvoegen
Laadpalen = pd.concat([Laadpalen, df5], axis=1)
Laadpalen.head()